In [29]:
import pandas as pd
import numpy as np

import inflection

In [3]:
df = pd.read_csv('..//zomato.csv')

In [9]:
df.columns

Index(['Restaurant ID', 'Restaurant Name', 'Country Code', 'City', 'Address',
       'Locality', 'Locality Verbose', 'Longitude', 'Latitude', 'Cuisines',
       'Average Cost for two', 'Currency', 'Has Table booking',
       'Has Online delivery', 'Is delivering now', 'Switch to order menu',
       'Price range', 'Aggregate rating', 'Rating color', 'Rating text',
       'Votes'],
      dtype='object')

In [22]:
df1 = df.copy()

In [34]:
COUNTRIES = {
    1: "India",
    14: "Australia",
    30: "Brazil",
    37: "Canada",
    94: "Indonesia",
    148: "New Zeland",
    162: "Philippines",
    166: "Qatar",
    184: "Singapure",
    189: "South Africa",
    191: "Sri Lanka",
    208: "Turkey",
    214: "United Arab Emirates",
    215: "England",
    216: "United States of America",
}

def country_name(country_id):
    return COUNTRIES[country_id]

def create_price_tye(price_range):
    if price_range == 1:
        return "cheap"
    elif price_range == 2:
        return "normal"
    elif price_range == 3:
        return "expensive"
    else:
        return "gourmet"
    
    
COLORS = {
    "3F7E00": "darkgreen",
    "5BA829": "green",
    "9ACD32": "lightgreen",
    "CDD614": "orange",
    "FFBA00": "red",
    "CBCBC8": "darkred",
    "FF7800": "darkred",
}
def color_name(color_code):
    return COLORS[color_code]

def rename_columns(dataframe):
    df = dataframe.copy()
    title = lambda x: inflection.titleize(x)
    snakecase = lambda x: inflection.underscore(x)
    spaces = lambda x: x.replace(" ", "")
    cols_old = list(df.columns)
    cols_old = list(map(title, cols_old))
    cols_old = list(map(spaces, cols_old))
    cols_new = list(map(snakecase, cols_old))
    df.columns = cols_new
    
    return df

df1 = rename_columns(df1);
df1['country'] = df1['country_code'].map(COUNTRIES);

In [66]:
df1.columns

Index(['restaurant_id', 'restaurant_name', 'country_code', 'city', 'address',
       'locality', 'locality_verbose', 'longitude', 'latitude', 'cuisines',
       'average_cost_for_two', 'currency', 'has_table_booking',
       'has_online_delivery', 'is_delivering_now', 'switch_to_order_menu',
       'price_range', 'aggregate_rating', 'rating_color', 'rating_text',
       'votes'],
      dtype='object')

In [80]:
# 1. Quantos restaurantes únicos estão registrados?
unique_restaurant = df1['restaurant_id'].nunique()

# 2. Quantos países únicos estão registrados?
countries = df1['country_code'].nunique()

# 3. Quantas cidades únicas estão registradas?
cities = df1['city'].nunique()

# 4. Qual o total de avaliações feitas?
votes = df1['votes'].sum()

# 5. Qual o total de tipos de culinária registrados?
cuisines = df1['cuisines'].nunique()


In [92]:
# 1. Qual o nome do país que possui mais cidades registradas?
cities_by_country = df1[['city', 'country']].groupby('country').count().sort_values('city', ascending=False).reset_index()

# 2. Qual o nome do país que possui mais restaurantes registrados?
restaurants_by_country = df1[['restaurant_id', 'country']].groupby('country').nunique().sort_values('restaurant_id', ascending=False).reset_index()

# 3. Qual o nome do país que possui mais restaurantes com o nível de preço igual a 4 registrados?
exp_rest_by_country = df1[['restaurant_id', 'country']].groupby('country').nunique().sort_values('restaurant_id', ascending=False).reset_index()

# 4. Qual o nome do país que possui a maior quantidade de tipos de culinária distintos?

# 5. Qual o nome do país que possui a maior quantidade de avaliações feitas?

# 6. Qual o nome do país que possui a maior quantidade de restaurantes que fazem entrega?

# 7. Qual o nome do país que possui a maior quantidade de restaurantes que aceitam reservas?

# 8. Qual o nome do país que possui, na média, a maior quantidade de avaliações registrada?

# 9. Qual o nome do país que possui, na média, a maior nota média registrada?

# 10. Qual o nome do país que possui, na média, a menor nota média registrada?

# 11. Qual a média de preço de um prato para dois por país?


In [90]:
def process_data(file_path):
    df = pd.read_csv(file_path)

    df = df.dropna()

    df = rename_columns(df)

    df["price_type"] = df.loc[:, "price_range"].apply(lambda x: create_price_tye(x))

    df["country"] = df.loc[:, "country_code"].apply(lambda x: country_name(x))

    df["color_name"] = df.loc[:, "rating_color"].apply(lambda x: color_name(x))

    df["cuisines"] = df.loc[:, "cuisines"].apply(lambda x: x.split(",")[0])

    df = df.drop_duplicates()

    df.to_csv("../dataset/data.csv", index=False)

    return df

In [91]:
process_data('../dataset/zomato.csv')

,restaurant_id,restaurant_name,country_code,city,address,locality,locality_verbose,longitude,latitude,cuisines,...,is_delivering_now,switch_to_order_menu,price_range,aggregate_rating,rating_color,rating_text,votes,price_type,country,color_name
0,6310675,Mama Lou's Italian Kitchen,162,Las Piñas City,"Block 1, Lot 36, Tropical Avenue Corner Tropic...",BF International,"BF International, Las Piñas City",121.009787,14.447615,Italian,...,0,0,3,4.6,3F7E00,Excellent,619,expensive,Philippines,darkgreen
2,6314542,Blackbird,162,Makati City,"Nielson Tower, Ayala Triangle Gardens, Salcedo...","Ayala Triangle Gardens, Salcedo Village, Makat...","Ayala Triangle Gardens, Salcedo Village, Makat...",121.024562,14.556042,European,...,0,0,4,4.7,3F7E00,Excellent,469,gourmet,Philippines,darkgreen
3,6301293,Banapple,162,Makati City,"Ayala Triangle Gardens, Salcedo Village, Makat...","Ayala Triangle Gardens, Salcedo Village, Makat...","Ayala Triangle Gardens, Salcedo Village, Makat...",121.023171,14.556196,Filipino,...,0,0,3,4.4,5BA829,Very Good,867,expensive,Philippines,green
4,6315689,Bad Bird,162,Makati City,"Hole In The Wall, Floor 4, Century City Mall, ...","Century City Mall, Poblacion, Makati City","Century City Mall, Poblacion, Makati City, Mak...",121.027708,14.565899,American,...,0,0,3,4.4,5BA829,Very Good,858,expensive,Philippines,green
5,6304833,Manam,162,Makati City,"Level 1, Greenbelt 2, Ayala Center, Greenbelt,...","Greenbelt 2, San Lorenzo, Makati City","Greenbelt 2, San Lorenzo, Makati City, Makati ...",121.020380,14.552351,Filipino,...,0,0,3,4.7,3F7E00,Excellent,930,expensive,Philippines,darkgreen
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
7522,5912546,Eataly,208,İstanbul,"Zorlu Center AVM, Köprü Katı, Levazım Mahalles...","Zorlu Center AVM, Levazım, Beşiktaş","Zorlu Center AVM, Levazım, Beşiktaş, İstanbul",29.017326,41.065322,Italian,...,0,0,4,4.3,5BA829,Very Good,1367,gourmet,Turkey,green
7523,5913006,Tarihi Çınaraltı Aile Çay Bahçesi,208,İstanbul,"Çengelköy Mahallesi, Çınaraltı Camii Sokak, No...",Çengelköy Merkez,"Çengelköy Merkez, İstanbul",29.052620,41.050280,Fast Food,...,0,0,2,4.5,3F7E00,Excellent,1172,normal,Turkey,darkgreen
7524,5923535,Boon Cafe & Restaurant,208,İstanbul,"Çengelköy Mahallesi, Çengelköy Caddesi, Kara S...",Çengelköy Merkez,"Çengelköy Merkez, İstanbul",29.052623,41.050717,Restaurant Cafe,...,0,0,4,4.2,5BA829,Very Good,1160,gourmet,Turkey,green
7525,5914190,Kanaat Lokantası,208,İstanbul,"Sultantepe Mahallesi, Selmani Pak Caddesi, No ...",Üsküdar Merkez,"Üsküdar Merkez, İstanbul",29.016590,41.025741,Home-made,...,0,0,3,4.0,5BA829,Very Good,770,expensive,Turkey,green


In [95]:
aux_map = df1[['latitude', 'longitude', 'aggregate_rating']].copy()
aux_map.rename( columns = {'latitude': 'lat', 'longitude': 'lon'}, inplace = True )
aux_map['aggregate_rating'] = aux_map['aggregate_rating']  * 1000
aux_map.head()

,lat,lon,aggregate_rating
0,14.447615,121.009787,4600.0
1,14.447615,121.009787,4600.0
2,14.556042,121.024562,4700.0
3,14.556196,121.023171,4400.0
4,14.565899,121.027708,4400.0
